# Describe the business problem

Travel agency business analyst (who) finds a decrease in revenue and determines that this is from a drop in repeat  business (what) and identifies the most profitable customers to target to book again. The marketing department wants to achieve more profitable results  by matching the right package for travel to each customer.

# A set of business constraints have to be respected:

We have a limited budget to run a marketing campaign
We want to determine which is the best way to contact the customers.
We need to identify which customers to contact


Prepare the data
Use decision optimization

    Step 1: Download the library
    Step 2: Set up the engines
    Step 3: Set up the prescriptive model
        Define the decision variables
        Set up the constraints
        Express the objective
        Solve with the Decision Optimization solve service
    Step 4: Analyze the solution and run an example analysis

Import Data from Object Storage

In [1]:
# The code was removed by DSX for sharing.

,CUST_ID,Traveling Party,Trip Delay,Destination Airport Code,Destination Continent,Destination Country,Destination City,Length of Trip,Destination Type,Booking Type,...,Month Num of Birth,Own Home,Responded to Promo,Sales Rep_ID,Age Band,Income_BIN,Return_Customer,$XF-Return_Customer,$XFC-Return_Customer,probability
0,1,2,Yes,AKL,Aust,New Zealand,Auckland,6,Hotel,Online,...,12,No,No,5,43 to 53,1,Y,N,0.870,0.130
1,1,1,No,DCA,NaN,USA,Washington DC,5,Resort,Online,...,12,No,No,5,43 to 53,1,Y,N,0.812,0.188
2,5,7,No,BGI,Caribbean,Barbados,Seawell,5,Hotel,Online,...,6,Yes,No,9,21_to 31,4,N,N,0.500,0.500
3,6,4,No,LAS,NaN,USA,Las Vegas,6,Timeshare,Online,...,1,Yes,No,7,21_to 31,4,Y,Y,0.826,0.826
4,7,4,No,CIA,EUR,Italy,Rome,10,Resort,Online,...,12,Yes,No,4,32 to 42,5,Y,Y,0.874,0.874


# Prepare Data

Replace spaces in column names and filter 2015 data and create Pivot table

In [2]:
df_data_1.columns = [c.replace(' ', '_') for c in df_data_1.columns]
df_data_1 = df_data_1[df_data_1.Booking_Year == 2015]
total_rows = df_data_1['CUST_ID'].count()
print (total_rows)

6270


In [3]:
PivotView = df_data_1.pivot_table(index='CUST_ID', columns = 'Package_Type', values='probability')

In [4]:
FilterData = PivotView.dropna(thresh=3)

In [5]:
list = []
for i in range(len(FilterData.index)):
    row = FilterData.iloc[[i]]
    tup = ((row.index.tolist()[0], row.columns[0], round(float(row[row.columns[0]].tolist()[0]),3), 
            row.columns[1], round(float(row[row.columns[1]].tolist()[0]),3),
            row.columns[2], round(float(row[row.columns[2]].tolist()[0]),3)))
    list.append(tup)
#list

In [6]:

df_data_2 = pd.read_csv(get_object_storage_file_with_credentials_05c30ea585c743be863f3cc490872dea('CPLEX', 'joined_trips_customers.csv'))
names = pd.DataFrame(df_data_2, columns = ['CUST_ID', 'Customer Name'])
names = names.rename(columns={'CUST_ID': 'customerid'})

Let's customize the display of this data and show the confidence forecast for each customer.

In [7]:
offers = pd.DataFrame(data=list, index=range(0, len(list)), columns=["customerid", "Package1", "Confidence1", "Package2", "Confidence2","Package3", "Confidence3"])
offers.head(5)

,customerid,Package1,Confidence1,Package2,Confidence2,Package3,Confidence3
0,735,Family,0.754,Outdoors,0.763,Sightseeing,0.769
1,1047,Family,0.765,Outdoors,0.761,Sightseeing,0.774
2,1315,Family,0.793,Outdoors,0.775,Sightseeing,0.804
3,3009,Family,0.862,Outdoors,0.850,Sightseeing,0.842
4,3285,Family,0.792,Outdoors,0.787,Sightseeing,0.799


In [8]:
packages = ["Family", "Outdoors", "Sightseeing"]
packageValue = [100, 200, 75]
budgetShare = [0.4, 0.4, 0.2]

availableBudget = 500
channels =  pd.DataFrame(data=[("email", 20.0, 0.20), ("newsletter", 15.0, 0.05), ("phone", 23.0, 0.20)], columns=["name", "cost", "factor"])

# Step 1: Download the library

First install docplex and set the credentials to solve the model using IBM ILOG CPLEX Optimizer on Cloud.


In [9]:
import docplex.mp

# Step 2: Set up the prescriptive engine

    Subscribe to the Decision Optimization on Cloud solve service here.
    Get the service URL and your personal API key and enter your credentials here:

In [10]:
url = "https://api-oaas.docloud.ibmcloud.com/job_manager/rest/v1/"
key = "api_312f8c64-4363-433b-a730-8ddcbdab3327"

# Step 3: Set up the prescriptive model
Create the model

In [11]:
from docplex.mp.model import Model

mdl = Model(name="marketing_campaign")

# Define the decision variables¶
The integer decision variables channelVars, represent whether or not a customer will be made an offer for a particular product via a particular channel.

In [12]:
offersR = range(0, len(offers))
packagesR = range(0, len(packages))
channelsR = range(0, len(channels))

The integer decision variable totaloffers represents the total number of offers made.

In [13]:
channelVars = mdl.binary_var_cube(offersR, packagesR, channelsR)
totaloffers = mdl.integer_var(lb=0)
budgetSpent = mdl.continuous_var()

The continuous variable budgetSpent represents the total cost of the offers made.

# Set up the constraints

    Offer only one product per customer.
    Compute the budget and set a maximum on it.
    Compute the number of offers to be made.

In [14]:
#Only 1 product is offered to each customer     
mdl.add_constraints( mdl.sum(channelVars[o,p,c] for p in packagesR for c in channelsR) <=1
                   for o in offersR)

mdl.add_constraint( totaloffers == mdl.sum(channelVars[o,p,c] 
                                           for o in offersR 
                                           for p in packagesR
                                           for c in channelsR) )

mdl.add_constraint( budgetSpent == mdl.sum(channelVars[o,p,c]*channels.get_value(index=c, col="cost") 
                                           for o in offersR 
                                           for p in packagesR
                                           for c in channelsR) )

# Balance the offers among products   
for p in packagesR:
    mdl.add_constraint( mdl.sum(channelVars[o,p,c] for o in offersR for c in channelsR) 
                       <= budgetShare[p] * totaloffers )

# Do not exceed the budget
mdl.add_constraint( mdl.sum(channelVars[o,p,c]*channels.get_value(index=c, col="cost") 
                            for o in offersR 
                            for p in packagesR 
                            for c in channelsR)  <= availableBudget )  
# mdl.print_information()


docplex.mp.linear.LinearConstraint[](20_x1+15_x2+23_x3+20_x4+15_x5+23_x6+20_x7+15_x8+23_x9+20_x10+15_x11+23_x12+20_x13+15_x14+23_x15+20_x16+15_x17+23_x18+20_x19+15_x20+23_x21+20_x22+15_x23+23_x24+20_x25+15_x26+23_x27+20_x28+15_x29+23_x30+20_x31+15_x32+23_x33+20_x34+15_x35+23_x36+20_x37+15_x38+23_x39+20_x40+15_x41+23_x42+20_x43+15_x44+23_x45+20_x46+15_x47+23_x48+20_x49+15_x50+23_x51+20_x52+15_x53+23_x54+20_x55+15_x56+23_x57+20_x58+15_x59+23_x60+20_x61+15_x62+23_x63+20_x64+15_x65+23_x66+20_x67+15_x68+23_x69+20_x70+15_x71+23_x72+20_x73+15_x74+23_x75+20_x76+15_x77+23_x78+20_x79+15_x80+23_x81+20_x82+15_x83+23_x84+20_x85+15_x86+23_x87+20_x88+15_x89+23_x90+20_x91+15_x92+23_x93+20_x94+15_x95+23_x96+20_x97+15_x98+23_x99+20_x100+15_x101+23_x102+20_x103+15_x104+23_x105+20_x106+15_x107+23_x108+20_x109+15_x110+23_x111+20_x112+15_x113+23_x114+20_x115+15_x116+23_x117+20_x118+15_x119+23_x120+20_x121+15_x122+23_x123+20_x124+15_x125+23_x126+20_x127+15_x128+23_x129+20_x130+15_x131+23_x132+20_x133+15_x134

Express the objective

We want to maximize the expected revenue.

In [15]:
mdl.maximize(
    mdl.sum( channelVars[idx,p,idx2] * c.factor * packageValue[p]* o.Confidence1  
            for p in packagesR 
            for idx,o in offers[offers['Package1'] == packages[p]].iterrows()  
            for idx2, c in channels.iterrows())
    +
    mdl.sum( channelVars[idx,p,idx2] * c.factor * packageValue[p]* o.Confidence2 
            for p in packagesR
            for idx,o in offers[offers['Package2'] == packages[p]].iterrows() 
            for idx2, c in channels.iterrows())
    +
    mdl.sum( channelVars[idx,p,idx2] * c.factor * packageValue[p]* o.Confidence3 
            for p in packagesR
            for idx,o in offers[offers['Package3'] == packages[p]].iterrows() 
            for idx2, c in channels.iterrows())
    )

Solve with the Decision Optimization solve service

In [16]:
s = mdl.solve(url=url, key=key)
print(s)

solution for: marketing_campaign
objective: 339.615
_x33=1
_x129=1
_x69=1
_x39=1
_x146=330.000
_x9=1
_x138=1
_x78=1
_x48=1
_x145=15
_x18=1
_x19=1
_x85=1
_x120=1
_x112=1
_x61=1
_x94=1



# Step 4: Analyze the solution¶

First, let's display the Optimal Marketing Channel per customer.

In [17]:
report = [(channels.get_value(index=c,col="name"), packages[p], offers.get_value(o, "customerid")) 
          for c in channelsR 
          for p in packagesR 
          for o in offersR  if channelVars[o,p,c].solution_value==1]
assert len(report) == totaloffers.solution_value

print("Marketing plan has {0} offers costing {1}".format(totaloffers.solution_value, budgetSpent.solution_value))

report_bd = pd.DataFrame(report, columns=['channel', 'product', 'customer'])
print(report_bd)

Marketing plan has 15.0 offers costing 330.0
   channel      product  customer
0    email       Family      1315
1    email     Outdoors      7279
2    email     Outdoors      9419
3    email     Outdoors     12208
4    email  Sightseeing      4528
5    phone       Family      3285
6    phone       Family      3750
7    phone       Family     13432
8    phone       Family     14625
9    phone       Family     14835
10   phone     Outdoors      3009
11   phone     Outdoors      4671
12   phone     Outdoors      4948
13   phone  Sightseeing       735
14   phone  Sightseeing      1047
